In [1]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings,elasticsearch
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
import os
from langchain_community.document_loaders import PyPDFLoader

from groq_api_key import groq_api_key
os.environ["GROQ_API_KEY"] = groq_api_key


# Function to process all PDFs in a folder
def load_and_split_pdfs(folder_path):
    all_pages = []

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            loader = PyPDFLoader(file_path)
            pages = loader.load_and_split()
            all_pages.extend(pages)
    
    return all_pages

# Example usage
folder_path = 'book_pdfs'
all_pages = load_and_split_pdfs(folder_path)

# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

texts = text_splitter.split_documents(all_pages)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

persist_directory = "Flamingo_db"

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

c:\Users\SASWAT\Documents\NCERT-BOT\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\SASWAT\Documents\NCERT-BOT\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# retriever
retriever = vectordb.as_retriever()

# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.6
)

# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [3]:
# invoke the qa chain and get a response for user query
query = "What doubts did Edla have about the peddler"
response = qa_chain.invoke({"query": query})
print(response['query'])
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

What doubts did Edla have about the peddler
Edla thought, "Either he has stolen something or else he has escaped from jail." She thought this as she saw the man's frightened expression and behavior.
******************************
Source Document: book_pdfs\lefl104.pdf


In [4]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Use the given context to answer the question. "
    "You are a highly knowledgeable NCERT assistant. "
    "Give answer in more than 200 words"
    "Your job is to provide accurate, detailed, and comprehensive answers based on the information retrieved from the NCERT textbooks. "
    "Please consider the educational context, relevant facts, and concepts while crafting your response. "
    "Make sure to explain the terms and concepts in simple language that a school student can understand. "
    "If you don't know the answer, say you don't know. "
    "Provide output in markdown format. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

persist_directory = "Flamingo_db"


# embeddings = HuggingFaceEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# load the chroma db
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

# retriever
retriever = vectordb.as_retriever()
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

human_query="Give me the summary of the chapter Deep Water in not less than 1000 words"
response=chain.invoke({"input": human_query})
print(response['answer'])
print(f"Number of words of the response :{response['answer'].count(' ')-1}")

**Deep Water: A Journey of Overcoming Fear**

The chapter "Deep Water" is an autobiographical account by William O. Douglas, a renowned American jurist and author. The narrative revolves around Douglas's childhood experience of nearly drowning in a swimming pool and his subsequent struggle to overcome the fear of water. The story is a poignant and introspective exploration of the human psyche, delving into the complexities of fear, courage, and personal growth.

**The Inciting Incident**

The story begins with Douglas recounting an incident from his childhood, when he was around ten or eleven years old. He had decided to learn how to swim, and the local YMCA pool in Yakima seemed like the perfect place to do so. However, his enthusiasm was short-lived, as he soon found himself struggling to stay afloat in the pool. Despite the presence of water wings, Douglas panicked and sank to the bottom of the pool. He recalls the sensation of being underwater, unable to breathe, and the feeling of

In [ ]:
human_query="Give me the summary of the chapter Deep Water in not less than 1000 words"
response=chain.invoke({"input": human_query})
print(response['answer'])
print(f"Number of words of the response :{response['answer'].count(' ')-1}")

**Deep Water: A Journey of Overcoming Fear**

The chapter "Deep Water" is an autobiographical account by William O. Douglas, a renowned American jurist and author. The narrative revolves around Douglas's childhood experience of nearly drowning in a swimming pool and his subsequent struggle to overcome the fear of water. Through this personal anecdote, Douglas explores the psychological aspects of fear, its impact on human behavior, and the importance of confronting and conquering it.

**The Inciting Incident**

The story begins with Douglas recounting an incident from his childhood when he was around ten or eleven years old. He had decided to learn how to swim and joined the YMCA in Yakima, where he lived. The YMCA pool was considered safe, with a gradual drop in depth from two to nine feet. Douglas obtained a pair of water wings and began his swimming lessons. However, during one of his early attempts, he experienced a terrifying moment that would leave a lasting impact on his life.

**The Traumatic Experience**

As Douglas was practicing his swimming strokes, he suddenly found himself submerged underwater, unable to breathe. He panicked and flailed at the surface, but his water wings were of little help. The experience was so traumatic that Douglas thought he was going to die. He recalls feeling a sense of resignation, as if he had accepted his fate. This near-death experience left an indelible mark on Douglas's psyche, instilling in him a deep-seated fear of water.

**The Fear Takes Hold**

The fear of water became an all-consuming force in Douglas's life. He would avoid any situation that involved water, including swimming, boating, or even wading in shallow waters. This phobia not only limited his recreational activities but also affected his self-esteem. Douglas felt embarrassed and ashamed of his fear, which he perceived as a sign of weakness. He became withdrawn and isolated, unable to share his feelings with anyone.

**The Turning Point**

Years later, when Douglas was in his mid-twenties, he decided to confront his fear head-on. He realized that his fear was not only limiting his personal growth but also affecting his relationships with others. Douglas began to take small steps towards overcoming his fear. He started by wading in shallow waters, gradually increasing the depth and duration of his exposure. He also practiced swimming strokes on land, building up his confidence and strength.

**The Triumph**

Douglas's determination and perseverance eventually paid off. He learned to swim and even began to enjoy the experience. He recalls a moment when he swam two miles across a lake, feeling a sense of exhilaration and accomplishment. This triumph marked a significant turning point in Douglas's life, as he had finally overcome his fear of water.

**The Larger Meaning**

Douglas's experience of overcoming his fear of water has a larger significance that extends beyond his personal life. He reflects on the nature of fear and its impact on human behavior. Douglas realizes that fear is a natural response to perceived threats, but it can also become a debilitating force that limits our potential. He quotes Franklin D. Roosevelt's famous phrase, "All we have to fear is fear itself," highlighting the importance of confronting and conquering fear.

**The Autobiographical Significance**

The chapter "Deep Water" is not only a personal account of Douglas's experience but also an exploration of the human psyche. Through his narrative, Douglas provides insight into the complexities of human behavior, revealing the ways in which fear can shape our lives. The autobiographical aspect of the chapter adds a layer of authenticity, making the story more relatable and engaging.

**The Themes**

The chapter "Deep Water" explores several themes that are relevant to human experience. Some of the major themes include:

1. **Fear and Anxiety**: The chapter highlights the debilitating effects of fear and anxiety on human behavior. Douglas's experience shows how fear can limit our potential and affect our relationships with others.
2. **Perseverance and Determination**: Douglas's story demonstrates the importance of perseverance and determination in overcoming fear and achieving personal growth.
3. **Self-Discovery**: The chapter is a journey of self-discovery, as Douglas reflects on his experiences and gains insight into his own psyche.
4. **Personal Growth**: The narrative shows how Douglas's experience of overcoming his fear of water leads to personal growth and a greater sense of confidence and self-esteem.

**The Style**

The chapter "Deep Water" is written in a reflective and introspective style, with Douglas narrating his experiences in the first person. The use of vivid imagery and descriptive language adds a layer of depth and emotion to the narrative, making the story more engaging and relatable. The chapter also includes quotes and references to other authors, adding a layer of complexity and depth to the narrative.

**The Significance**

The chapter "Deep Water" is significant not only for its autobiographical value but also for its exploration of the human psyche. The narrative provides insight into the complexities of human behavior, highlighting the ways in which fear can shape our lives. The chapter also demonstrates the importance of perseverance and determination in overcoming fear and achieving personal growth. As a literary work, "Deep Water" is a powerful and engaging narrative that explores the human condition in a way that is both relatable and thought-provoking.
Number of words of the response :842

In [30]:
# response = qa_chain.invoke({"query": query})
# print(response["result"])
# print("*"*30)
# print("Source Document:", response["source_documents"][0].metadata["source"])

According to the text, Edla had the following doubts about the peddler:

"Either he has stolen something or else he has escaped from jail", she thought.

This suggests that Edla suspected the peddler of being a thief or a fugitive.
******************************
Source Document: book_pdfs\lefl104.pdf


In [5]:
response

{'input': 'Give me the summary of the chapter Deep Water in not less than 1000 words',
 'context': [Document(metadata={'page': 6, 'source': 'book_pdfs\\lefl103.pdf'}, page_content='Deep Water /29\nWentworth in New Hampshir e, dived of f a dock at T riggs\nIsland, and swam two miles across the lake to Stamp Act\nIsland. I swam the crawl, breast stroke, side stroke, and\nback stroke. Only once did the terror return. When I was\nin the middle of the lake, I put my face under and saw\nnothing but bottomless water . The old sensation r eturned\nin miniatur e. I laughed and said, “W ell, Mr T error, what do\nyou think you can do to me?” It fled and I swam on.\nYet I had r esidual doubts. At my first opportunity I\nhurried west, went up the T ieton to Conrad Meadows, up\nthe Conrad Cr eek T rail to Meade Glacier , and camped in\nthe high meadow by the side of W arm Lake. The next\nmorning I stripped, dived into the lake, and swam across\nto the other shore and back — just as Doug Corpron used